In [1]:
# TODO: replace df with dataframe
# TODO: investigate presence of multiple 'Plate ID' columns in condition_summary
# TODO: add dataset selector (or make new notebooks for analyzing different datasets - this might be better, think about it.)

In [2]:
execfile('./src/display_utils.py')
execfile('./src/moldev_utils.py')
%matplotlib inline
from scipy.stats.kde import gaussian_kde
import scipy.signal as signal
from toolz import thread_last
from scipy.stats import ks_2samp

:0: FutureWarning: IPython widgets are experimental and may change in the future.


In [3]:
cell_path = '/notebooks/moldev-data/analyzed/07-20-2015/ssC-cells.csv'

In [4]:
raw_cell_data = pd.read_csv(cell_path)

In [5]:
raw_cell_data['Cell Type'] = raw_cell_data['Condition'].map(lambda string: string.split(' ')[-1])

# Add cell phase information

In [6]:
# DataFrame -> String -> (a | [a] | Series[a]) -> DataFrame
def add_col(dataframe,colname,values):
    "Add column to dataframe with given values."
    dataframe[colname] = values
    return dataframe

In [7]:
# [Float] -> Float
def get_threshold(intensity,bw = 0.35):
    """ Given an array of intensities, find the intensity 
        threshold separating the G1 peak from the S/G2 peak. """
    kde_fit = gaussian_kde(intensity,bw)
    x = np.linspace(min(intensity),
                    max(intensity),
                    1000)
    y = kde_fit(x)
    ddx = np.diff(np.diff(y)) # second derivative of fit
    peak = np.argmin(ddx) # most frequent intensity 
    index = min([yi for yi in signal.argrelmax(ddx)[0] if yi > peak])
    threshold = x[index]
    return threshold

In [8]:
def label_cell_phase(dataframe):
    col = 'NucIntegrated Intensity_Avg'
    threshold = get_threshold(dataframe[col])
    dataframe['Cell Phase'] = dataframe[col].map(
        lambda intensity: 'G1' if intensity < threshold else 'S/G2')
    dataframe['Cell Phase Threshold'] = threshold
    dataframe['Condition'] = dataframe['Condition'] + ' ' + dataframe['Cell Phase']
    return dataframe

In [9]:
def label_cell_phase_2(dataframe_orig):
    dataframe = dataframe_orig.copy()
    col = 'NucIntegrated Intensity_Avg'
    threshold = get_threshold(dataframe[col])
    dataframe['Cell Phase'] = dataframe[col].map(
        lambda intensity: 'G1' if intensity < threshold else 'S/G2')
    dataframe['Cell Phase Threshold'] = threshold
    dataframe['Condition'] = dataframe['Cell Type'] + ' ' + dataframe['Cell Phase']
    return dataframe

In [10]:
phased_conditions = thread_first(
    raw_cell_data,
    lambda x: x.groupby('Condition'),
    lambda xs: map(snd,xs),
    lambda conditions: map(label_cell_phase,conditions))

In [11]:
phased_conditions_by_cell_type = thread_first(
    raw_cell_data,
    lambda x: x.groupby('Cell Type'),
    lambda xs: map(snd,xs),
    lambda conditions: map(label_cell_phase_2,conditions))

In [12]:
by_cell_type = thread_last(
    raw_cell_data,
    lambda x: x.groupby('Cell Type'),
    lambda x: map(snd,x),
    (map,lambda x: add_col(x,'Condition',x['Cell Type'])))

In [13]:
# mydf = pd.DataFrame({'a':[1,2,3],'b':[10,20,30],'c':[100,200,300]})

# def f(x):
#     print x
#     return x['a'] * 2
# mygroups = mydf.groupby(f,axis=0)

In [14]:
# phased_conditions_by_cell_type = thread_first(
#     raw_cell_data,
#     lambda x: x.groupby('Cell Type'),
#     lambda xs: map(snd,xs),
#     lambda conditions: map(label_cell_phase_2,conditions))

In [15]:
phased_conditions[0].head()[['Condition','Cell Phase','Cell Phase Threshold']]

,Condition,Cell Phase,Cell Phase Threshold
30921,ssC 0.08 % Media Control 143b S/G2,S/G2,19329075.431431
30922,ssC 0.08 % Media Control 143b S/G2,S/G2,19329075.431431
30923,ssC 0.08 % Media Control 143b S/G2,S/G2,19329075.431431
30924,ssC 0.08 % Media Control 143b S/G2,S/G2,19329075.431431
30925,ssC 0.08 % Media Control 143b S/G2,S/G2,19329075.431431


In [16]:
thresholded_data = pd.concat(phased_conditions)
thresholded_data_by_cell_type = pd.concat(phased_conditions_by_cell_type)
all_by_cell_type = pd.concat(by_cell_type)

KeyboardInterrupt: 

In [ ]:
thresholded_data.to_csv('/notebooks/tmp/1.csv',index=False)
thresholded_data_by_cell_type.to_csv('/notebooks/tmp/2.csv',index=False)
all_by_cell_type.to_csv('/notebooks/tmp/3.csv',index=False)
raw_cell_data.to_csv('/notebooks/tmp/4.csv')

In [ ]:
raw_cell_data = pd.concat([all_by_cell_type,thresholded_data_by_cell_type,raw_cell_data, thresholded_data])

In [ ]:
df1 = pd.read_csv('/notebooks/tmp/1.csv')
df2 = pd.read_csv('/notebooks/tmp/2.csv')
df3 = pd.read_csv('/notebooks/tmp/3.csv')
df4 = pd.read_csv('/notebooks/tmp/4.csv')

In [ ]:
raw_cell_data = pd.concat([df1,df2,df3,df4])

In [ ]:
raw_cell_data.to_csv('/notebooks/moldev-data/analyzed/07-20-2015/ssC-phase_split-cells.csv',index=False)